# <center> Econ 577: Quantitative Economics (with Python) </center>
## <center> Homework 7 </center>

# <font color='red'>Name:</font>

In [34]:
#libraries that will be used in this HW 
import os
import shutil as sh
import pandas as pd
#you can add other libraries as needed
import numpy as np
import datetime
import matplotlib.pylab as plt
%matplotlib inline 
import scipy.optimize as so
from scipy.interpolate import interp1d


In [8]:
# To find your working directory:
%pwd 
# Code in case you want to change your working directory:  %cd
# for example: %cd "C:\Users\xinxi\Dropbox\Spring2022\ECON590\HW1"
# Define your data folder here:
datafolder = "C:\\Users\\xinxi\\Dropbox\\Spring2022\\ECON590\\HW7\\\HW7_Data\\"
outputfolder = "C:\\Users\\xinxi\\Dropbox\\Spring2022\\ECON590\\HW7\\\HW7_Output\\"
# Please use an absolute path

Consider the following variation of the finite horizon inter-temporal consumptions and saving problem discussed in the lecture.

You own a share of stock that pays a dividend $D_t\in\{1,5\}$ each period. Specifically, the dividend realization depends on how well the company does: when the company does well you get $5$ and when the company does poorly you get $1$. Assume that how well the company does is driven by the state of the economy and that the economic transition is modeled by the  Markov Chain process:

$$S \in \{Expansion,Recession\}$$

$$M=\left(\begin{array}{cc} 
.8 & .2\\ 
.4 & .6
\end{array}\right)$$

That is, when the economy is in the Expansion, the dividend is 5, but when the economy is in the recession, the dividend is 1.

Suppose that you begin with $x$ in your savings. Further suppose your utility of consuming $c$ units in a period is given by $u(c)=\sqrt{c}$, and that you live for $T=10$ periods.

In addition, suppose that the assets held at the end of period $t$ yield an interest (with interest rate $r=.05$) that is paid at the beginning.

Investigate the role of the initial starting balance in your savings. Specifically, plot $x \in [0,10]$ on the x-axis and average total lifetime utility on the y-axis.

# Question 1

Step 1: What should you do in period 10? -- spend everything

In this step, define a function, max_asset, to calculate the maximumal asset/wealth possible at period T:
    - input: the starting asset (a1), the maximal income in each period (I), interest rate (r), the number of periods (T)
    - output: a number 
    Hint: max_asset(0,5,0.05,10) = 62.889462677744156


Step 2: Suppose you are in period 9 and you have A=10 in total,and you are in expension, what you are maximizing is: 
$$ u(x)+ .8(Value10((A-x)*1.05+5)+.2(Value10((A-x)*1.05+1)$$

Now, suppose you are in period 9 and you have A=10 in total,and you are in recession, what are you maximizing? 

Write a function named negExpectedPayoff9 to calculate your utility in either expansion or recession. 
    - input: your consumption (x), your asset (A), your state( S, 0 for Expansion and 1 for Recession)
    - output: the corresponding utility 
    Hint: negExpectedPayoff9(x,A,0) = u(x)+ .8(Value10((A-x)*1.05+5)+.2(Value10((A-x)*1.05+1)

<br>

**In your submission write two functions *max_asset* and *negExpectedPayoff***

Here is the outline of how you should structure your code for this question in your submission.

```
def max_asset(a1, I, r, T)
    #...
    return #a number

def negExpectedPayoff9(x, A, state)
    #...
    return #utility
```



In [4]:
def max_asset(a1, I, r, T):

    a_T = 0
    
    for i in range(T):
        a_T = (a1 * (1+r)) + I #multiplies assets by interest rate over each period
        a1 = a_T
     
    return a_T

def negExpectedPayoff9(x, A, state):
    
    def util(x):
        return np.sqrt(x)
    
    NPOINTS = 15

    assets10=np.linspace(0,max_asset(0,5,.05,10),NPOINTS)
    policy10=assets10.copy()
    value10=util(policy10)

    V10 = interp1d(assets10, value10, kind='cubic')
    
    T = np.array([[.8,.2],[.4,.6]])
    
    if state == 0:
        return -(util(x) + T[state][0]*V10((A-x)*1.05 + 5) + T[state][1]*V10((A-x)*1.05 + 1))
    else:
        return -(util(x) + T[state][0]*V10((A-x)*1.05 + 5) + T[state][1]*V10((A-x)*1.05 + 1))



62.889462677744156

# Question 2

Step 3: Find the optimal policy in period 9 (i.e., for all possible A)

Hint: your policy and value functions should reflect the two states.

Step 4: Find the optimal policy for every period t (continue working backwards)

<br>

**In your submission wrap you code for this question in a function called q2() which takes no arguments and returns your answer for step 3 and step 4**

Here is the outline of how you should structure your code for this question in your submission. Make sure you return your answers the same order as below in your submission:

```
def q2():
    # ...
    
    return optimal_policy_period9, optimal_policies
```



In [42]:
def q2():
    
    NPOINTS = 15
    
    def util(x):
        return np.sqrt(x)

    assets9=np.linspace(0,max_asset(0,5,.05,10),NPOINTS)
    policy9=np.zeros((1,NPOINTS,2))
    value9=np.zeros((1,NPOINTS,2))

    for state in [0,1]:
        
        for i,A in enumerate(assets9):

            if i==0:
                policy9[0][i][state] = 0
                value9[0][i][state] = - negExpectedPayoff9(0,0,state)
            else:        
                res = so.minimize(negExpectedPayoff9,A/2,args=(A,state)) #Notice we pass extra parameters to expectedPayoffs via args=(.)
                policy9[0][i][state] = res.x[0]
                value9[0][i][state] = -res.fun

    optimal_policy_period9 = policy9
    
    def negExpectedPayoff(x,A,S,Vexp,Vrec):
        T = np.array([[.8,.2],[.5,.5]])
        return -(util(x)+T[S][0]*Vexp(A-x+10)+T[S][1]*Vrec(A-x))

    T=10

    assets = np.zeros(shape=(T,NPOINTS)) 
    policies = np.zeros(shape=(T,NPOINTS,2)) #The third entry corresponds to states of the economy
    values = np.zeros(shape=(T,NPOINTS,2))

    for t in range(T,0,-1):

        assets[t-1] = np.linspace(0,max_asset(0,5,.05,t),NPOINTS)

        if t==T:

            #Expansion state
            policies[t-1,:,0] = assets[t-1].copy() #the best policy in period 20 is to spend everything
            values[t-1,:,0] = util(policies[t-1,:,0]) #the value of having x units is u(x)

            #Recession state
            policies[t-1,:,1] = assets[t-1].copy() #the best policy in period 20 is to spend everything
            values[t-1,:,1] = util(policies[t-1,:,1]) #the value of having x units is u(x)

        else:

            VnextExp = interp1d(assets[t], values[t,:,0], kind='cubic',fill_value="extrapolate")
            VnextRec = interp1d(assets[t], values[t,:,1], kind='cubic',fill_value="extrapolate")

            for s in [0,1]: #need to go through expansion (0) and recession (1) states

                for i,A in enumerate(assets[t-1]):

                    if i==0:

                        policies[t-1,i,s] = 0
                        values[t-1,i,s] = - negExpectedPayoff(0,0,s,VnextExp,VnextRec)

                    else:

                        bnds = ((0,A),) #set boundaries for the optimizer
                        res = so.minimize(negExpectedPayoff,A/2,args=(A,s,VnextExp,VnextRec),bounds=bnds) #Notice we pass extra parameters to expectedPayoffs via args=(.)
                        policies[t-1,i,s] = res.x[0]
                        values[t-1,i,s] = - res.fun
                        
    optimal_policies = policies

    return optimal_policy_period9, optimal_policies


# Question 3

Simulate 1000 times to calculate the average total lifetime utility for each starting balance x.

<br>

**In your submission wrap you code for this question in a function called q3() which takes no arguments and returns None.**

**This function should save a plot called [username]_q3.pdf (replace [username] with your purdue login) to folder hw11_q3_output which has the starting balances x on the x-axis and the average total lifetime utility on the y-axis. Axis labels are required for full points.**

Here is the outline of how you should structure your code for this question in your submission.

```
def q3():
    # ...

    import os
    
    # os.path.join is an operating system independent way to create a path to a file in a directory: e.g. on a Mac it results in 'hw11_q3_output/[username]_q3.pdf'
    plt.savefig(os.path.join('hw11_q3_output', '[username]_q3.pdf'))
    
    return None
```



In [44]:
def q3():
    
    def util(x):
        return np.sqrt(x)
    
    T = 10
    policies = q2()[1]
  
    print(type(policies[0][2]))
    import os
    
    NSIM = 10000

    np.random.seed(0) #set seed so results are replicable
    income = np.random.choice([1,5],p=[.5,.5],size=(10,NSIM)) #pre-draw NSIM income streams
    results = np.empty(NSIM) #allocate space for the results (i.e., utilities for each of NSIM simulations)

    
    for sim in range(NSIM):

        balance = 2
        utility = 0

        for t in range(T):
            balance += income[t,sim]
            spend = policies[t][balance].mean #I know my code breaks here but I can't figure out why, it should return the avg  
            utility += util(spend)       #of an array with each element corresponding to the opt policy for each state
            balance -= spend
            balance = balance*1.05

        results[sim]=utility
    
    plt.hist(results)
    plt.title('Distribution of Lifetime Utility')
    plt.plot([results.mean(),results.mean()],[0,2000],'red',label='average')
    plt.legend()
    plt.show()

    plt.savefig(os.path.join('hw7_q3_output', 'mart2336_q3.pdf'))
    
    
    return None


<class 'numpy.ndarray'>


IndexError: arrays used as indices must be of integer (or boolean) type